$$\textrm{Joaquin Peñuela Parra}$$
$$\textrm{University of Los Andes}$$
$$\textrm{High Energy Physics Group: Phenomenology of Particles}$$

This code was written to be running in Docker. If you do not have a Docker inside hep-server2 please refer to: https://github.com/Phenomenology-group-uniandes/Tutoriales_Generales

$\textbf{Preliminaries}$ 

The libraries used here are:

In [ ]:
import numpy as np
import pandas as pd
import glob
import os

In [ ]:
background = ["stop", "w_jets", "z_jets", "ww", "wz", "zz", "ttbar"]
personal_folder="/disco4/personal_folders/Joaquin"

In [ ]:
#testeo
#background = ["z_jets", "zz"]

In [ ]:
for process in background:
    nombre_script = f"{personal_folder}/SM_Backgrounds/Scripts/{process}.txt"
    !./../../../Collider/MG5_aMC_v3_1_0/bin/mg5_aMC {nombre_script}

In [ ]:
xs = []
for i in range(len(background)):
    process = background[i]
    xs_process = []

    folder = f"{personal_folder}/SM_Backgrounds/Data/{process}"
    
    files = glob.iglob(os.path.join(folder, "Events/**/tag_1_merged_xsecs.txt"))    
    for file_path in files:
        if os.path.isfile(file_path):
            table = np.loadtxt(file_path, skiprows=1)
            for i in range(len(table)):
                if(table[i,0] == 45):
                    xs_process.append(table[i,1])
                    
    if (len(xs_process) != 0):
        xs.append(np.mean(np.asarray(xs_process)))
    else:
        file_path = f"{folder}/crossx.html"
        tabla = pd.read_html(file_path)
        columna_xs = tabla[0]['Cross section (pb)']
        for k in range(len(columna_xs)):
            xs_process.append(float(columna_xs[k].split(' ')[0]))
        xs.append(np.mean(np.asarray(xs_process)))
        
xs_tabla = np.transpose(pd.DataFrame(xs))

xs_github = np.loadtxt("SimulationsPaths.txt",dtype='str',delimiter=',')
xs = []
for i in range(len(background)):
    for j in range(len(xs_github)):
        if(background[i] == xs_github[j][0]):
            xs.append(float(xs_github[j][2]))
            break
            
xs_tabla_github = np.transpose(pd.DataFrame(xs))

xs = np.concatenate((xs_tabla,xs_tabla_github))

razon = np.asmatrix(xs)[0,:]/np.asmatrix(xs)[1,:]
xs = np.concatenate((xs, razon))

xs = pd.DataFrame(xs)
xs.columns = background
xs.index = ["XS (13.6 TeV): " , "XS (13.0 TeV): ", "K:"]
xs.to_excel('/disco4/personal_folders/Joaquin/SM_Backgrounds/XS_Matriz.xlsx')